---
# TD2 Clustering par la méthode k-means
## - Application à la gestion des risques -
---

### Objectif pédagogique

À l'issue de ce TD, l'étudiant devra être capable de :

- Comprendre le principe mathématique de l'algorithme k-means ;
- Implémenter et utiliser k-means en Python avec *`scikit-learn`* ;
- Choisir un nombre pertinent de clusters (méthode du coude, silhouette) ;
- Interpréter des clusters dans un contexte de gestion des risques ;
- Discuter les limites de k-means pour des applications réelles de gestion du risque.

### Contexte applicatif : gestion des risques naturels – incendies de forêt et WUI

On se place dans le contexte de la gestion des risques naturels, et plus particulièrement des incendies de forêt à l’interface forêt–habitat (Wildland–Urban Interface, WUI).

Les territoires WUI sont caractérisés par une forte hétérogénéité des conditions de risque, liée à la végétation, à la topographie, au climat et à l’exposition des enjeux humains et matériels.

L’objectif est d’utiliser un algorithme de clustering non supervisé (k-means) pour :

- Identifier des profils homogènes de zones WUI ;
- Mettre en évidence des niveaux de risque différenciés ;
- Aider à la priorisation des actions de prévention (débroussaillement, urbanisme, moyens de lutte).
---

### Partie 1 : Génération d'un jeu de données de risque incendie (WUI)

On considère un territoire découpé en 300 zones WUI, caractérisées par les indicateurs suivants :

*`fuel_load`* : charge combustible (kg/m²) ;

*`slope`* : pente moyenne du terrain (°) ;

*`wind_exposure`* : exposition moyenne au vent (m/s) ;

*`ignition_freq`* : fréquence annuelle d’ignitions ;

*`asset_value`* : valeur des enjeux exposés (M€).

Ces variables sont inspirées des modèles physiques et empiriques utilisés en science du feu et en analyse de vulnérabilité territoriale.

#### 1.1 Simulation des données

- On génère 300 situations de terrain (pixels SIG, zones, mailles, scénarios).

- Elles sont volontairement réparties en 3 groupes de taille égale :

    - Risque faible
    - Risque modéré
    - Risque élevé
        
🎯 Objectif pédagogique

👉 Construire un jeu de données contrôlé, où les clusters existent réellement, afin de :

- tester K-means,
- analyser la séparabilité,
- interpréter physiquement les résultats.

In [1]:
import numpy as np
import pandas as pd

#--------------------------------------------------------------------------------------------------------------------------
# Data generation
#--------------------------------------------------------------------------------------------------------------------------

# Creation d'un dataFrame avec les variables en colonnes et les observations en lignes
# on ajoute chaque cluster a une liste 
# puis on concatene la liste en un seul DataFrame
# on impose une structure latente avec les 3 clusters, K-means doit les retrouver

n = 300

cluster_sizes = [100, 100, 100]

data_list = []

# Cluster 0 : risque faible
data_list.append(pd.DataFrame({
    "fuel_load": np.random.lognormal(mean=0.6, sigma=0.2, size=cluster_sizes[0]),
    "slope": np.random.uniform(0, 10, size=cluster_sizes[0]),
    "wind_exposure": np.random.gamma(1.0, 1.0, size=cluster_sizes[0]),
    "ignition_freq": np.random.gamma(1.0, 0.5, size=cluster_sizes[0]),
    "asset_value": np.random.lognormal(mean=1.5, sigma=0.3, size=cluster_sizes[0]),
}))

# Cluster 1 : risque modéré
data_list.append(pd.DataFrame({
    "fuel_load": np.random.lognormal(mean=1.2, sigma=0.3, size=cluster_sizes[1]),
    "slope": np.random.uniform(10, 25, size=cluster_sizes[1]),
    "wind_exposure": np.random.gamma(2.0, 1.5, size=cluster_sizes[1]),
    "ignition_freq": np.random.gamma(1.5, 1.0, size=cluster_sizes[1]),
    "asset_value": np.random.lognormal(mean=2.2, sigma=0.4, size=cluster_sizes[1]),
}))

# Cluster 2 : risque élevé
data_list.append(pd.DataFrame({
    "fuel_load": np.random.lognormal(mean=1.8, sigma=0.3, size=cluster_sizes[2]),
    "slope": np.random.uniform(25, 40, size=cluster_sizes[2]),
    "wind_exposure": np.random.gamma(3.0, 2.0, size=cluster_sizes[2]),
    "ignition_freq": np.random.gamma(2.5, 1.5, size=cluster_sizes[2]),
    "asset_value": np.random.lognormal(mean=3.0, sigma=0.6, size=cluster_sizes[2]),
}))

data = pd.concat(data_list, ignore_index=True)

1. Appliquer la méthode K-means afin de retrouver les clusters

2. Tracer le Scatterplot *`fuel_load`* vs *`ignition_freq`* en colorant les clusters

3. Tracer le Pairplot de toutes les variables

4. Afin de faciliter l'interprétation des histogrammes (diagonale de la matrice Pairplot), on peut tracer pour chaque cluster, les densités de chaque variable (transformation des histogrammes en densités continues) avec la fonction *`kdeplot`* du module Python *`seaborn`*

5. Afin de faire l'analyse de risque, il faut revenir dans l'espace physique.   
    - On peut partitionner le DataFrame selon la valeur de la colonne *`cluster`* à l'aide de la méthode *`groupeby()`* de *`panda`* et ensuite calculer la moyenne de chaque variable à l'aide de la méthode *`mean()`* de *`panda`*.
    - On tracera enseuite sur le même graphe les 3 histogrammes des 5 variables pour chaque cluster. Avec un modèle de risque, on peut classifier les clusters par niveau de risque.

6. On calculera un score de risque comme étant le produit des variables contributives.  
On tracera la densité du score de risque par cluster 